**Import PyVisa and check if installed**

In [10]:
import pyvisa
print(pyvisa.__version__)

1.14.1


In [11]:
# Initialize the VISA resource manager
rm = pyvisa.ResourceManager()

# Open a connection to the GPIB instrument
# Replace 'GPIB0::INSTR' with the appropriate GPIB address for your instrument
# For example, 'GPIB0::5::INSTR' where 5 is the address of the instrument
gpib_address = 'GPIB0::5::INSTR'
instrument = rm.open_resource(gpib_address)

# Send a command to the probe station
# Replace 'YOUR_COMMAND' with the actual command you want to send
command = 'YOUR_COMMAND'
instrument.write(command)

# If you need to read a response from the instrument
response = instrument.read()
print("Response from the instrument:", response)

# Close the connection
instrument.close()

ValueError: Please install linux-gpib (Linux) or gpib-ctypes (Windows, Linux) to use this resource type. Note that installing gpib-ctypes will give you access to a broader range of functionalities.
No module named 'gpib'

In [19]:
def parse_gcode_line(line):
    # Initialize variables with default values
    gCommand = None
    gX = None
    gY = None
    gZ = None

    # Split the line into parts
    parts = line.split()

    # Iterate through each part to find the command and coordinates
    for part in parts:
        if part.startswith('G'):
            gCommand = part
        elif part.startswith('X'):
            gX = float(part[1:])  # Convert to float and ignore the 'X' character
        elif part.startswith('Y'):
            gY = float(part[1:])  # Convert to float and ignore the 'Y' character
        elif part.startswith('Z'):
            gZ = float(part[1:])  # Convert to float and ignore the 'Z' character

    return gCommand, gX, gY, gZ

def process_gcode_file(filename):
    with open(filename, 'r') as file:
        for line in file:
            # Strip comments and whitespace
            line = line.split(';')[0].strip()

            # Skip empty lines
            if not line:
                continue

            # Parse the line
            gCommand, gX, gY, gZ = parse_gcode_line(line)

            # Print or store the result
            print(f"GCommand: {gCommand}, X: {gX}, Y: {gY}, Z: {gZ}")
            #HERES MY SEND OVER TCP FUNCTION
            
            
def send_command_line(command_to_send)
    #Create dictionary to switch between commands and send them over the ports using 
    #Send Via communicate_with_instrument_via_tcp(ip_address, port, command_to_send)
    
def communicate_with_instrument_via_tcp(ip_address, port, command):
    # Initialize the VISA resource manager
    rm = pyvisa.ResourceManager()

    # Construct the TCP/IP address string
    resource_name = f"TCPIP0::{ip_address}::{port}::SOCKET"

    try:
        # Open a connection to the instrument
        instrument = rm.open_resource(resource_name)
        print(f"Connected to {resource_name}")

        # Send the command to the instrument
        instrument.write(command)
        print(f"Sent command: {command}")

        # Optionally read the response from the instrument
        response = instrument.read()
        print(f"Received response: {response}")

    except pyvisa.VisaIOError as e:
        print(f"VISA I/O error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the connection
        instrument.close() # NEED TO FIX IF NO DEVICE THEN THIS LINE FAULTS CAUSE THERES NO "instrument" device
        print("Connection closed.")

# Example usage
if __name__ == "__start__":
    gcode_file = 'example.gcode'  # Replace with your GCODE file path
    process_gcode_file(gcode_file)
    # Specify the IP address and port of the instrument
    ip_address = "192.168.1.100"  # Replace with your instrument's IP address
    port = 5025                   # Replace with your instrument's port

    # The command you want to send PUT THIS IN A CASE STATEMENT FOR GCODE TRANSLATION
    #command_to_send = "*IDN?"  # Common command to query the instrument identity

    # Call the function to communicate with the instrument
    communicate_with_instrument_via_tcp(ip_address, port, command_to_send)